# Matrix Spectral Factorization
Here we employ the Chandrasekhar-Kailath-Morf-Sidhu (CKMS) Filter. This process requires the specral density to be in the form of a Laurent polynomial (or the ratio of Laurent polynomials). So, the first thing to do will be to fit a Laurent polynomial to the function.

In [12]:
using LinearAlgebra
using FFTW
using StatsBase

using Plots
pyplot()

include("SFbyCKMS_Matrix.jl")

function z_crossspect(X,Y,L = 50, Nex = 2^10; win = "Par")
    ## Y = d x 1 x steps, X = nu x 1 x steps
    d, stepsx = size(X)
    nu, stepsy = size(Y)

    Nexh = Int(floor(Nex/2))
    lags = -L:L;

    stepsx == stepsy || print("X and Y are not the same length. Taking min.")
    steps = minimum([stepsx stepsy])

    # Smoothed viewing window
    if win == "Bar"
        lam = 1 .- (0:L)/L
    elseif win == "Tuk"
        lam = .5*(1 .+ cos.(pi/L*(0:L)))
    elseif win == "Par"
        LL = Int(floor(L/2))
        lam1 = 1 .- 6*((0:LL)/L).^2 .+ 6*((0:LL)/L).^3
        lam2 = 2*(1 .- (LL+1:L)/L).^3
        lam = [lam1; lam2]
    else
        lam = ones(L+1)
    end
    Lam = [lam[L+1:-1:2]; lam]

    C_smoothed = zeros(d,nu,length(lags))
    for i = 1 : d
        for j = 1 : nu
            C_smoothed[i,j,:] = Lam .* crosscov(X[i,1:steps],Y[j,1:steps],lags)
        end
    end

    ## C_smoothed = d x nu x 2L+1

    ## Pad with zeros in preparation for fft
    C_padded = cat(dims = 3, zeros(d,nu,Nex - Nexh - L), C_smoothed, zeros(d,nu,Nexh - L - 1))
    C = fftshift(C_padded,3)

    z_crossspect_num_fft = fft(C,3);
end

function Matrix_CKMS_c_test(P)
    d = size(P)[1];
    m = size(P)[3] - 1

    NN = reverse(P[:,:,2:end],dims = 3)
    Re = Rr = p0 = P[:,:,1]

    F = [[zeros(d,d*(m-1)); I] zeros(d*m,d)]
    h = [zeros(d,d*(m-1)) I]

    K = zeros(d*m,d)
    for i = 0 : m-1
        K[d*i + 1: d*(i+1),:] = NN[:,:,i+1]
    end
    L = K

    for i = 1:200
        hL = h*L
        FL = F*L

        K_new = K - FL/Rr*hL'
        L_new = FL - K/Re*hL
        Re_new = Re - hL/Rr*hL'
        Rr_new = Rr - hL'/Re*hL

        K = K_new
        L = L_new
        Re = Re_new
        Rr = Rr_new
    end

    k = K/Re
    re = Re

    sqrt_re = sqrt(re)
    
    l = zeros(d,d,m)
    l[:,:,1] = sqrt_re;
    for i = m-1:-1:0
        l[:,:,m-i] = k[d*i + 1: d*(i+1),:]*sqrt_re
    end
    l
end

Matrix_CKMS_c_test (generic function with 1 method)

In [13]:
# Graph stuff
Nex = 1000;
Theta = 2pi*(0:Nex)/Nex;
Z = map(th -> exp(im*th),Theta);

Here is the model:
$$X_t = \Phi_1X_{t-1} + A_t$$
where
$$\Phi_1 = \begin{pmatrix} r_1 + \gamma & \gamma\\
r_1 + \gamma - r_2 & r_2 - \gamma \end{pmatrix} \qquad \text{and}\qquad A_t \sim N(0,R)$$
Here we choose $r_1 = -0.8$, $r_2 = 0.6$, and $\gamma = 0.4$. and $R = I$. 

The $z$-spectrum of this is 
$$S_X(z) = \frac{25z}{(3z - 5)(4z + 5)(5z - 3)(5z + 4)}
\begin{pmatrix} 
5(z^2 - 6z + 1) & 25z^2 - z + 10 \\
10z^2 - z + 25  & 2(5z^2 + 27z + 5) \\
\end{pmatrix}$$

This has known factorization of 
$$L(z) = \begin{pmatrix}1 - 0.4z^{-1} & 0.4z^{-1}\\
z^{-1} & 1-0.2z^{-1}\end{pmatrix}$$

In [14]:
# Model stuff
steps = 10^6
discard = 10^4

10000

In [ ]:
# VAR(1) Process
Phi1 = [-0.4 0.4; -1 0.2]

S_X_minus_ana(z) = inv(I - Phi1.*z^(-1))

S_X_ana(z) = S_X_minus_ana(z)*S_X_minus_ana(z)'

X = zeros(2, steps + discard)
A = randn(2, steps + discard)

for i = 2 : steps + discard
    X[:,i] = Phi1*X[:,i-1] + A[:,i]
end
X = X[:,discard + 1: steps + discard]

plot(X[:,100:200]')

## MA(1) Diagonal

In [ ]:
# # MA(1) Diagonal process
# r1, r2 = 1 .- 2*rand(2)
# Mu1 = [r1 0; 0 r2]

# S_X_minus_ana(z) = I - Mu1.*z^(-1)

# S_X_ana(z) = S_X_minus_ana(z)*S_X_minus_ana(z)'

# X = zeros(2, steps + discard)
# A = randn(2, steps + discard)

# for i = 2 : steps + discard
#     X[:,i] = A[:,i] - Mu1*A[:,i-1]
# end
# X = X[:,discard + 1: steps + discard]

# plot(X[:,100:200]')

## MA(1) Nondiagonal

In [ ]:
# # MA(1) Diagonal process
# r1, r2 = 1 .- 2*rand(2)
# alph = randn()
# Mu1 = [r1 0; alph r2]

# S_X_minus_ana(z) = I - Mu1.*z^(-1)

# S_X_ana(z) = S_X_minus_ana(z)*S_X_minus_ana(z)'

# X = zeros(2, steps + discard)
# A = randn(2, steps + discard)

# for i = 2 : steps + discard
#     X[:,i] = A[:,i] - Mu1*A[:,i-1]
# end
# X = X[:,discard + 1: steps + discard]

# plot(X[:,100:200]')

In [ ]:
L = 60
S_X_num = z_spect(X,L; win = "Par")

S_X_num_plot = complex(zeros(2,2,Nex))
for n = 1 : Nex
    S_X_num_plot[:,:,n] = S_X_num(Z[n])
end

S_X_ana_plot = complex(zeros(2,2,Nex))
for n = 1 : Nex
    S_X_ana_plot[:,:,n] = S_X_ana(Z[n])
end

plot(Theta, [S_X_num_plot[1,1,:] S_X_num_plot[2,1,:] S_X_num_plot[1,2,:] S_X_num_plot[2,2,
            :] S_X_ana_plot[1,1,:] S_X_ana_plot[2,1,:] S_X_ana_plot[1,2,:] S_X_ana_plot[2,2,:]])

## Estimating the spectral factor using CKMS

In [ ]:
L = 60
lags = 0:L

LL = Int(floor(L/2))
lam1 = 1 .- 6*((0:LL)/L).^2 .+ 6*((0:LL)/L).^3
lam2 = 2*(1 .- (LL+1:L)/L).^3
lam = [lam1; lam2]

P = Autocov(X,lags);

In [ ]:
l = Matrix_CKMS_c(P);

l_test = Matrix_CKMS_c_test(P)

S_X_minus_num(z) = sum([l[:,:,i+1]*z^(-i) for i = 0 : length(l)-1])

In [ ]:
S_X_num_sf(z) = S_X_minus_num(z)*S_X_minus_num(z)'

S_X_num_sf_plot = complex(zeros(2,2,Nex))
for n = 1 : Nex
    S_X_num_sf_plot[:,:,n] = S_X_num_sf(Z[n])
end

plot(Theta, [S_X_num_sf_plot[1,1,:] S_X_num_sf_plot[1,2,:] S_X_num_sf_plot[2,2,
            :] S_X_ana_plot[1,1,:] S_X_ana_plot[1,2,:] S_X_ana_plot[2,2,:]])

## Test a simple process